In [31]:
# Imports
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.layers import TextVectorization
import tensorflow_datasets as tfds
from bs4 import BeautifulSoup   # Removes html tags from strings
import string

In [32]:
# testing out the tokenizer 
sentences = [
    'Hello, my friend!',
    'Goodbye, my brother.'
]

test_sentences = [
    'Hi, how are you my friend?',
    'My sister, goodbye!'
]

tokenizer = Tokenizer(num_words = 100, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
word_index

{'<OOV>': 1, 'my': 2, 'hello': 3, 'friend': 4, 'goodbye': 5, 'brother': 6}

In [33]:
sequences = tokenizer.texts_to_sequences(sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
print(sequences)
print(test_sequences)

[[3, 2, 4], [5, 2, 6]]
[[1, 1, 1, 1, 2, 4], [2, 1, 5]]


In [34]:
# padding
padded = pad_sequences(test_sequences, maxlen=10)
print(padded)

[[0 0 0 0 1 1 1 1 2 4]
 [0 0 0 0 0 0 0 2 1 5]]


In [35]:
table2 = str.maketrans('', '', string.punctuation)
table2

{33: None,
 34: None,
 35: None,
 36: None,
 37: None,
 38: None,
 39: None,
 40: None,
 41: None,
 42: None,
 43: None,
 44: None,
 45: None,
 46: None,
 47: None,
 58: None,
 59: None,
 60: None,
 61: None,
 62: None,
 63: None,
 64: None,
 91: None,
 92: None,
 93: None,
 94: None,
 95: None,
 96: None,
 123: None,
 124: None,
 125: None,
 126: None}

In [36]:
# remove stopwords
stopwords = ["a", "about", "above", "yours", "yourself", "yourselves"]

# load in dataset, preprocess, and index words
table = str.maketrans('', '', string.punctuation)
imdb_sentences = []
train_data = tfds.as_numpy(tfds.load('imdb_reviews', split="train"))
for item in train_data:
    sentence = str(item['text'].decode('UTF-8').lower())
    # preprocess 
    sentence = sentence.replace(",", " , ")
    sentence = sentence.replace(".", " . ")
    sentence = sentence.replace("-", " - ")
    sentence = sentence.replace("/", " / ")
    soup = BeautifulSoup(sentence)
    sentence = soup.get_text()
    words = sentence.split()
    filtered_sentence = ""
    for word in words:
        word = word.translate(table)
        if word not in stopwords:
            filtered_sentence = filtered_sentence + word + " "
    imdb_sentences.append(filtered_sentence)
tokenizer = Tokenizer(num_words=25000)
tokenizer.fit_on_texts(imdb_sentences)
sequences = tokenizer.texts_to_sequences(imdb_sentences)
tokenizer.word_index

/var/folders/wg/bqtpsjds1p16_p2clkzkw5d80000gn/T/ipykernel_55573/3622997077.py:15: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sentence)


{'the': 1,
 'and': 2,
 'of': 3,
 'to': 4,
 'is': 5,
 'in': 6,
 'it': 7,
 'i': 8,
 'this': 9,
 'that': 10,
 'was': 11,
 'as': 12,
 'for': 13,
 'with': 14,
 'movie': 15,
 'but': 16,
 'film': 17,
 'on': 18,
 'not': 19,
 'you': 20,
 'are': 21,
 'his': 22,
 'have': 23,
 'be': 24,
 'he': 25,
 'one': 26,
 'its': 27,
 'all': 28,
 'at': 29,
 'by': 30,
 'an': 31,
 'they': 32,
 'so': 33,
 'who': 34,
 'from': 35,
 'like': 36,
 'her': 37,
 'or': 38,
 'just': 39,
 'out': 40,
 'has': 41,
 'if': 42,
 'some': 43,
 'there': 44,
 'what': 45,
 'good': 46,
 'more': 47,
 'when': 48,
 'very': 49,
 'up': 50,
 'no': 51,
 'time': 52,
 'she': 53,
 'even': 54,
 'my': 55,
 'would': 56,
 'which': 57,
 'only': 58,
 'story': 59,
 'really': 60,
 'see': 61,
 'their': 62,
 'were': 63,
 'had': 64,
 'can': 65,
 'me': 66,
 'well': 67,
 'than': 68,
 'we': 69,
 'much': 70,
 'been': 71,
 'bad': 72,
 'get': 73,
 'will': 74,
 'do': 75,
 'also': 76,
 'into': 77,
 'people': 78,
 'other': 79,
 'great': 80,
 'because': 81,
 'first'

In [37]:
sequences2 = tokenizer.texts_to_sequences(test_sentences)
sequences2

[[6655, 83, 21, 20, 55, 457], [55, 787, 6816]]

In [46]:
reverse_word_index = dict([(value, key) for (key, value) in tokenizer.word_index.items()])
decoded = []
for i in range(len(sequences2)):
    decoded_review = ' '.join([reverse_word_index.get(j, '?') for j in sequences2[i]])
    decoded.append(decoded_review)
decoded

['hi how are you my friend', 'my sister goodbye']

In [47]:
# New dataset 

(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k',
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    as_supervised=True,
    with_info=True
)

2023-06-29 18:35:09.086867: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /Users/samin/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete2QVMN1/imdb_reviews-test.tf…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /Users/samin/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete2QVMN1/imdb_reviews-unsuper…

Dataset imdb_reviews downloaded and prepared to /Users/samin/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [ ]:
encoder = info.features['text'].encoder
print('Vocabulary size: {}'.format(encoder.vocab_size))
print(encoder.subwords)